In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [12]:
# 获取数据集
train_dataset = datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

In [13]:
batch_size = 64

In [14]:
# 设置数据加载器
train_data_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_data_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [15]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1,6,kernel_size=5,stride=1,padding=2),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.ReLU(),
            nn.Conv2d(6,16,kernel_size=5,stride=1,padding=2),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(16*7*7,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
        )
    
    def forward(self,x):
        y = self.layers(x)
        y = y.view(y.size()[0],-1)
        y = self.fc_layers(y)
        return y

In [16]:
# 实例化模型
model = Net()
# 定义损失函数和优化器
optimizer = optim.Adam(model.parameters(), lr=0.01)
ce_loss = nn.CrossEntropyLoss()

In [17]:
# 查看模型参数
for name,para in model.named_parameters():
    print(name,para.shape)

layers.0.weight torch.Size([6, 1, 5, 5])
layers.0.bias torch.Size([6])
layers.3.weight torch.Size([16, 6, 5, 5])
layers.3.bias torch.Size([16])
fc_layers.0.weight torch.Size([120, 784])
fc_layers.0.bias torch.Size([120])
fc_layers.2.weight torch.Size([84, 120])
fc_layers.2.bias torch.Size([84])
fc_layers.4.weight torch.Size([10, 84])
fc_layers.4.bias torch.Size([10])


In [18]:
# 完成所有批次的轮询，就代表一次训练的epoch结束
def train():
    model.train()
    for _,data in enumerate(train_data_loader):
        data,label = data
        label = F.one_hot(label,num_classes=10).float()
        pred = model(data)     
        # 注意如果最后使用cross entropy loss，则不需要softmax层，直接使用pred即可
        # 但是如果使用其他loss，则需要先softmax，再计算loss
        loss = ce_loss(pred,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

In [19]:
# 测试集数据准确率
def test():
    model.eval()
    correct = 0
    total = 0
    for _,data in enumerate(test_data_loader):
        data,label = data
        y_pred = model(data)
        y_pred_label = torch.argmax(y_pred,dim=1)
        correct += torch.eq(y_pred_label,label).sum().item()
        total += data.shape[0]
    print('Test Accuracy: {:.2f}%'.format(100*correct/total))
    
    correct = 0
    total = 0
    for _,data in enumerate(train_data_loader):
        data,label = data
        y_pred = model(data)
        y_pred_label = torch.argmax(y_pred,dim=1)
        correct += torch.eq(y_pred_label,label).sum().item()
        total += data.shape[0]
    print('Train Accuracy: {:.2f}%'.format(100*correct/total))


In [20]:
for epoch in range(5):
    print("Epoch:{:2d}".format(epoch+1))
    train()
    correct_prob = test()
    print("="*20)

Epoch: 1
Test Accuracy: 97.75%
Train Accuracy: 97.96%
Epoch: 2
Test Accuracy: 97.72%
Train Accuracy: 97.94%
Epoch: 3
Test Accuracy: 96.75%
Train Accuracy: 97.05%
Epoch: 4
Test Accuracy: 97.82%
Train Accuracy: 98.59%
Epoch: 5
Test Accuracy: 97.96%
Train Accuracy: 98.58%
